In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os

import numpy as np
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from keras.callbacks import EarlyStopping, LearningRateScheduler

from keras.layers import Dense, Dropout
from keras.layers import Flatten, Conv1D, LeakyReLU, BatchNormalization, AveragePooling1D, MaxPooling1D
from keras.layers import LSTM

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from contextlib import redirect_stdout

In [0]:
# paths
directory = "drive/My Drive/UE_Proell"
preprocessed = f"{directory}/preprocessed"
result = f"{directory}/result"

# hyper parameter
loss = 'categorical_crossentropy'
learning_rate = 0.01
leaky_alpha = 0.01
dropout = 0.3
batch_size = 40
epochs = 250

# configuration parameter
save_model_infs = False
validation_split = 0.2
patience = 20
verbose = 1
seed = 42

# labels
labels = [
    "air conditioner",
    "car horn",
    "children playing",
    "dog bark",
    "drilling",
    "engine idling",
    "gun shot",
    "jackhammer",
    "siren",
    "street music"
]

# others
model_name = "urbaner"

In [0]:
os.makedirs(result, exist_ok=True)

X_train, y_train = np.load(f"{preprocessed}/X_train.npy"), np.load(f"{preprocessed}/y_train.npy")
X_test, y_test = np.load(f"{preprocessed}/X_test.npy"), np.load(f"{preprocessed}/y_test.npy")

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=validation_split, random_state=seed, stratify=y_train)

num_of_labels = y_train.shape[1]

print("X_train", X_train.shape, "-", "y_train", y_train.shape)
print("X_valid", X_valid.shape, "-", "y_valid", y_valid.shape)
print("X_test", X_test.shape, "-", "y_test", y_test.shape)

In [0]:
def getDNN(input_shape):
  model = Sequential()

  model.add(Dense(512, input_shape=input_shape, activation='relu'))
  model.add(Dropout(dropout))

  model.add(Dense(256, activation='relu'))
  model.add(Dropout(dropout))

  model.add(Dense(num_of_labels, activation='softmax'))
  return model

In [0]:
def getCNN(input_shape):
    model = Sequential()
    model.add(Conv1D(filters = 1, kernel_size = 3, input_shape=input_shape))
    model.add(LeakyReLU(alpha = leaky_alpha))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dropout(dropout))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_of_labels, activation='softmax'))
    return model

In [0]:
def getLSTM(input_shape):
  model = Sequential()
  model.add(LSTM(32, input_shape=input_shape))
  model.add(Dropout(dropout))
  model.add(Dense(num_of_labels, activation='softmax'))
  return model

In [0]:
networks = {
    "dnn": lambda: getDNN((40, )),
    "cnn": lambda: getCNN((40, 1)),
    "lstm": lambda: getLSTM((1, 40))
}

In [0]:
def plot_confusion_matrix(confusion_matrix, class_names, **kwargs):
  df_cm = pd.DataFrame(confusion_matrix, index = class_names, columns = class_names)
  plt.figure(figsize = (10,7))
  sns.heatmap(df_cm, annot=True, cmap=kwargs["cmap"])
  plt.savefig(f"{kwargs['result_path']}/confusion_matrix.png", dpi=400)
  plt.close()

In [0]:
def save_model_information(network, hist, scores, y_test_indices, y_pred_indices):
  result_path = f"{result}/{network}"
  if not os.path.exists(result_path):
    os.makedirs(result_path)

  model.save(f"{result_path}/{model_name}")

  # plot history for accuracy
  plt.figure()
  plt.ioff()
  plt.plot(hist.history['acc'])
  plt.plot(hist.history['val_acc'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'validation'], loc='upper left')
  plt.savefig(f"{result_path}/accuracy_{model_name}")

  # plot history for loss
  plt.figure()
  plt.ioff()
  plt.plot(hist.history['loss'])
  plt.plot(hist.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'validation'], loc='upper left')
  plt.savefig(f"{result_path}/loss_{model_name}")

  # generate other metrics + plot confusion matrixs
  confusion_matrix = metrics.confusion_matrix(y_test_indices, y_pred_indices)
  report           = metrics.classification_report(y_test_indices, y_pred_indices, target_names=labels, digits=3)
  plot_confusion_matrix(confusion_matrix, labels, cmap="Blues", result_path=result_path)

  # save detail information
  with open(f"{result_path}/{model_name}.txt","w+") as f:
      f.write('Trainingparameter:\n')
      f.write('ValidationSplit: {0}\n'.format(validation_split))
      f.write('Batchsize: {0}\n'.format(batch_size))
      f.write('Epochs: {0}\n'.format(epochs))
      f.write('Patience for Early Stopping: {0}\n'.format(patience))
      f.write('\n')
      f.write('Results:\n')
      f.write('Trainingaccuracy: {0}\n'.format(hist.history['acc'][-1]*100))
      f.write('Testaccuracy: {0}\n'.format(scores[1] * 100))
      f.write('\n')
      f.write('Confusion matrix\n')
      f.write(str(confusion_matrix))
      f.write('\n\n')
      f.write("Classification report\n")
      f.write(report)
      f.write("\n")
      f.write('Modelarchitecture:\n')
      with redirect_stdout(f):
          model.summary()

In [0]:
for network, model_function in networks.items():
  print(f"[{network}] Train network")

  model = model_function()
  model.compile(Adam(lr=learning_rate), loss=loss, metrics=['accuracy'])

  callbacks = [
        EarlyStopping(monitor="loss", patience=patience, verbose=0, mode="auto"),
  ]
  
  X = {
      "train": X_train.copy(),
      "test": X_test.copy(),
      "valid": X_valid.copy()
  }

  if network == "cnn":
    X["train"] = np.expand_dims(X["train"], axis=2)
    X["test"] = np.expand_dims(X["test"], axis=2)
    X["valid"] = np.expand_dims(X["valid"], axis=2)

  if network == "lstm":
    X["train"] = np.expand_dims(X["train"], axis=1)
    X["test"] = np.expand_dims(X["test"], axis=1)
    X["valid"] = np.expand_dims(X["valid"], axis=1)

  print(f"Current shape for input values: {X['train'].shape}")

  hist = model.fit(X["train"], y_train, validation_data=(X["valid"], y_valid), batch_size=batch_size, epochs=epochs, shuffle=True, verbose=verbose, callbacks=callbacks)
  
  scores = model.evaluate(X["test"], y_test, verbose=0)
  print(f"[{network}] Testaccuracy {scores[1] * 100}")

  y_pred_indices = np.argmax(model.predict(X["test"]), axis=1)
  y_test_indices = np.argmax(y_test, axis=1)

  if save_model_infs:
    save_model_information(network, hist, scores, y_test_indices, y_pred_indices)